In [2]:
import numpy as np
import pandas as pd


## 🔹 1. Simple Randomization (Coin Flip)

- Each unit (user/cluster) is independently randomized.
    * May result in imbalance (e.g., 7 treatment, 3 control).
	* Fine when large samples are available.

In [3]:

n_clusters = 10
assignment = np.random.choice([0, 1], size=n_clusters)
print(assignment)


[0 0 1 0 1 1 0 1 0 0]


## 🔹 2. Complete Randomization (Fixed Split)

- Ensure exact balance between groups.
- Guarantees equal treatment/control sizes (5 and 5 here).
- Common in A/B tests to avoid imbalance.

In [4]:
n_clusters = 5
half = n_clusters // 2

# Randomly decide which label gets the extra cluster
labels = [0, 1]
np.random.shuffle(labels)  # randomize label order

assignment = np.array([labels[0]]*half + [labels[1]]*(n_clusters - half))
np.random.shuffle(assignment)  # shuffle to randomize cluster assignment
print(assignment)

[0 1 1 1 0]


## 🔹 3. Blocked Randomization

Randomize within small blocks to maintain balance over time or within subgroups.
- Every 4 clusters → 2 control, 2 treatment.
- Prevents imbalance if experiment stops early.


In [5]:
n_clusters = 12
block_size = 4   # each block has 2 control, 2 treatment
assignment = []

for _ in range(n_clusters // block_size):
    block = np.array([0] * (block_size//2) + [1] * (block_size//2))
    np.random.shuffle(block)
    assignment.extend(block)

print(assignment)

[np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]


## 🔹 4. Stratified Randomization

First divide clusters into strata (based on covariates), then randomize within each stratum.

Example: Stratify clusters into “small” vs “large” based on size.
- Ensures treatment/control are balanced within strata.
- Useful if cluster size (or baseline metric) strongly influences outcome.

In [6]:


np.random.seed(42)
cluster_sizes = np.random.randint(50, 100, size=10)  # cluster sizes

df = pd.DataFrame({"cluster": range(10), "size": cluster_sizes})

df["stratum"] = np.where(df["size"] < 75, "small", "large")

assignments = []
for stratum, group in df.groupby("stratum"):
    n = len(group)
    assign = np.array([0]*(n//2) + [1]*(n - n//2))
    np.random.shuffle(assign)
    assignments.extend(assign)

df["treatment"] = assignments
print(df)

   cluster  size stratum  treatment
0        0    88   large          1
1        1    78   large          0
2        2    64   small          0
3        3    92   large          1
4        4    57   small          0
5        5    70   small          1
6        6    88   large          1
7        7    68   small          0
8        8    72   small          1
9        9    60   small          0


## 🔹 5. Re-Randomization / Constrained Randomization

### Simple re-randomization of the cluster-sizes
Randomize, then check balance on covariates. If imbalance is too high, randomize again.
- Guarantees covariate balance.
- Used in constrained randomization designs.

In [26]:
def randomize_until_balanced(cluster_sizes, threshold=5):
    n = len(cluster_sizes)
    while True:
        assignment = np.random.choice([0, 1], size=n)
        mean_diff = abs(cluster_sizes[assignment==1].mean() -
                        cluster_sizes[assignment==0].mean())
        if mean_diff < threshold:  # accept only if balanced
            return assignment

np.random.seed(42)
sizes = np.random.randint(50, 100, size=10)
assign = randomize_until_balanced(sizes, threshold=3)
print("Cluster sizes:", sizes)
print("Assignment:", assign)

Cluster sizes: [88 78 64 92 57 70 88 68 72 60]
Assignment: [1 1 1 0 0 0 0 0 0 1]


### Multi-variate re-rendomization
🧠 1️⃣ What it is

In single-covariate rerandomization (like your code), you only check one balance condition:

|mean(cluster size in treatment) − mean(cluster size in control)| ≤ threshold

But in real experiments, you often have several baseline covariates to balance:
- cluster size
- baseline engagement rate
- region
- average income
- etc.

You want all of them to be “well balanced” between treatment and control.

So you extend the rerandomization criterion to a vector of covariates.

⸻

⚙️ 2️⃣ How it’s done: the Mahalanobis distance approach

You compute a single multivariate balance metric that summarizes how different the two groups are across all covariates simultaneously.

That metric is the Mahalanobis distance between the treatment and control covariate means:

M = (\bar{x}_T - \bar{x}_C)’ \Sigma^{-1} (\bar{x}_T - \bar{x}_C)

where
- \bar{x}_T, \bar{x}_C = vectors of mean covariate values in treatment and control
- \Sigma = covariance matrix of covariates across all units

Then you:
1.	Randomize assignments.
2.	Compute M.
3.	If M < \text{threshold}, accept; otherwise, rerandomize.

This ensures overall covariate balance across all dimensions, not just one.

⸻


In [ ]:
import numpy as np
import pandas as pd

# --------------------------
# Step 1: Define Mahalanobis distance
# --------------------------
def mahalanobis_distance(df_treat, df_ctrl, covariate_cols):
    # Ensure all covariates are numeric
    x_t = df_treat[covariate_cols].astype(float).to_numpy()
    x_c = df_ctrl[covariate_cols].astype(float).to_numpy()
    
    # Compute mean difference vector
    diff = x_t.mean(axis=0) - x_c.mean(axis=0)
    
    # Combine treatment + control to compute covariance
    combined = np.vstack([x_t, x_c])  # shape = (n_rows, n_covariates)
    
    # Compute covariance matrix across columns (covariates)
    cov = np.cov(combined, rowvar=False)  # rowvar=False => columns = variables

    # Invert covariance
    inv_cov = np.linalg.inv(cov)
    
    # Mahalanobis distance
    return diff.T @ inv_cov @ diff

# --------------------------
# Step 2: Rerandomization loop
# --------------------------
def rerandomize_df(df, covariate_cols, threshold=0.3, max_iter=10000):
    n = len(df)
    for _ in range(max_iter):
        # Random assignment 0=control, 1=treatment
        df["treatment"] = np.random.choice([0, 1], size=n)
        df_treat = df[df["treatment"] == 1]
        df_ctrl = df[df["treatment"] == 0]
        
        # Compute Mahalanobis distance
        M = mahalanobis_distance(df_treat, df_ctrl, covariate_cols)
        
        # Accept assignment if within threshold
        if M < threshold:
            return df.copy(), M
    raise RuntimeError("Failed to find balanced randomization")

# --------------------------
# Step 3: Simulate cluster data
# --------------------------
np.random.seed(42)
df = pd.DataFrame({
    "cluster_id": range(100),
    "cluster_size": np.random.randint(50, 100, size=100),
    "baseline_rate": np.random.uniform(0.05, 0.15, size=100),
    "region": np.random.choice(["NA", "EU", "APAC"], size=100)
})

# Convert categorical variable 'region' to numeric dummies
df = pd.get_dummies(df, columns=["region"], drop_first=True)

# --------------------------
# Step 4: Run rerandomization
# --------------------------
covariates = ["cluster_size", "baseline_rate", "region_EU", "region_NA"]

balanced_df, M_final = rerandomize_df(df, covariates, threshold=0.3)
print(f"Accepted assignment with Mahalanobis distance: {M_final:.4f}")

# --------------------------
# Step 5: Check covariate balance
# --------------------------
summary = balanced_df.groupby("treatment")[covariates].mean().T
summary["abs_diff"] = abs(summary[0] - summary[1])
print(summary)

[[ 2.08732424e+02  3.76027701e-02 -8.89898990e-01  1.35454545e-01]
 [ 3.76027701e-02  7.33238792e-04  3.14143334e-04 -9.11409806e-04]
 [-8.89898990e-01  3.14143334e-04  2.12121212e-01 -1.12121212e-01]
 [ 1.35454545e-01 -9.11409806e-04 -1.12121212e-01  2.35454545e-01]]
Accepted assignment with Mahalanobis distance: 0.0615
treatment              0          1  abs_diff
cluster_size   72.826087  75.129630  2.303543
baseline_rate   0.096849   0.099722  0.002872
region_EU       0.326087   0.277778  0.048309
region_NA       0.391304   0.351852  0.039452


 ## 🔹 6. Cluster Randomization 

Instead of randomizing individuals, you randomize entire clusters (groups of users).
- Example: In a social network, you don’t want friends in different arms because of interference. So you randomize at the cluster (friend group) level.- - Another example: Randomize whole schools into “new curriculum” vs “old curriculum.”

Steps:
1.	Identify clusters (e.g., schools, groups).
2.	Randomly assign whole clusters to treatment/control.
3.	All individuals in a cluster follow the cluster’s assignment.

In [8]:
import numpy as np
import pandas as pd

# Suppose we have 10 clusters (schools), each with multiple students
np.random.seed(42)
students = pd.DataFrame({
    "student_id": np.arange(1, 51),
    "cluster_id": np.repeat(np.arange(1, 11), 5)  # 10 clusters, 5 students each
})

# Randomly assign clusters to treatment/control
clusters = students["cluster_id"].unique()
treatment_clusters = np.random.choice(clusters, size=len(clusters)//2, replace=False)

# Mark assignment at cluster level
students["assignment"] = students["cluster_id"].apply(
    lambda c: "Treatment" if c in treatment_clusters else "Control"
)

print(students.head(15))

    student_id  cluster_id assignment
0            1           1  Treatment
1            2           1  Treatment
2            3           1  Treatment
3            4           1  Treatment
4            5           1  Treatment
5            6           2  Treatment
6            7           2  Treatment
7            8           2  Treatment
8            9           2  Treatment
9           10           2  Treatment
10          11           3    Control
11          12           3    Control
12          13           3    Control
13          14           3    Control
14          15           3    Control


## Stratified Block Randomization

This combines:
- Stratification: ensures balance across important covariates (e.g., gender, age group).
- Blocking: ensures balance within small blocks.

This is especially common in clinical trials where you want guaranteed balance across multiple dimensions.

Example:
- Suppose we want equal Treatment/Control assignment within each gender group.
- Within each stratum (Male/Female), we also randomize in blocks of size 4 to keep things balanced.

In [9]:
import numpy as np
import pandas as pd

np.random.seed(123)
# Simulated participants
participants = pd.DataFrame({
    "id": np.arange(1, 21),
    "gender": np.random.choice(["Male", "Female"], size=20)
})

def stratified_block_randomization(df, stratify_col, block_size=4):
    assignments = []
    for stratum, group in df.groupby(stratify_col):
        group_ids = group["id"].tolist()
        np.random.shuffle(group_ids)
        
        # Break into blocks
        for i in range(0, len(group_ids), block_size):
            block = group_ids[i:i+block_size]
            # Assign half to treatment, half to control
            half = len(block) // 2
            assignments.extend([(pid, "Treatment") for pid in block[:half]])
            assignments.extend([(pid, "Control") for pid in block[half:]])
    
    return pd.DataFrame(assignments, columns=["id", "assignment"])

assignments = stratified_block_randomization(participants, "gender", block_size=4)
participants = participants.merge(assignments, on="id")

print(participants.sort_values("id"))

    id  gender assignment
0    1    Male    Control
1    2  Female    Control
2    3    Male    Control
3    4    Male    Control
4    5    Male  Treatment
5    6    Male    Control
6    7    Male  Treatment
7    8  Female    Control
8    9  Female  Treatment
9   10    Male  Treatment
10  11  Female  Treatment
11  12  Female  Treatment
12  13    Male    Control
13  14  Female    Control
14  15    Male    Control
15  16  Female  Treatment
16  17    Male  Treatment
17  18  Female    Control
18  19  Female    Control
19  20    Male  Treatment


# --------------------------------------------------------------------------------
# Practice

## 🔹 1. Simple Randomization (Coin Flip)

- Each unit (user/cluster) is independently randomized.
    * May result in imbalance (e.g., 7 treatment, 3 control).
	* Fine when large samples are available.

In [10]:
sample_size = 10
labels = np.random.choice([0, 1], size=sample_size)
labels

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

## 🔹 2. Complete Randomization (Fixed Split)

- Ensure exact balance between groups.
- Guarantees equal treatment/control sizes (5 and 5 here).
- Common in A/B tests to avoid imbalance.

In [11]:
sample_size = 10
half = sample_size // 2

labels = np.array([0 , 1])
np.random.shuffle(labels)

assignments = np.array([labels[0]]* half + [labels[1]]* (sample_size - half))
np.random.shuffle(assignments)
assignments

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

## 🔹 3. Blocked Randomization

Randomize within small blocks to maintain balance over time or within subgroups.
- Every 4 clusters → 2 control, 2 treatment.
- Prevents imbalance if experiment stops early.


In [12]:
sample_size = 100
block_size = 10
labels = np.array([0,1])
half = block_size // 2
assignments = []
for _ in range(sample_size // block_size):
    np.random.shuffle(labels)
    block = np.array([labels[0]] * half + [labels[1]] * (block_size - half))
    np.random.shuffle(block)
    assignments.extend(block)
assignments[0:10]


[np.int64(1),
 np.int64(0),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(0)]

## 🔹 4. Stratified Randomization

First divide clusters into strata (based on covariates), then randomize within each stratum.

Example: Stratify clusters into “small” vs “large” based on size.
- Ensures treatment/control are balanced within strata.
- Useful if cluster size (or baseline metric) strongly influences outcome.

In [13]:
np.random.seed(42)
num_clusters = 30
cluster_sizes = np.random.randint(50, 100,  size=num_clusters)
df = pd.DataFrame({"cluster":range(num_clusters),
                   "size": cluster_sizes
                   })
df["stratum"] = np.where(df['size'] > 75, "large", "small" )
grp_labels = [0, 1]
assigments = []
for startum, group in df.groupby("stratum"):
    np.random.shuffle(grp_labels)
    grp_size = len(group)
    print(grp_size)
    half = grp_size // 2
    labels = np.array([grp_labels[0]] * half + [grp_labels[1]] * (grp_size - half) )
    np.random.shuffle(labels)
    assigments.extend(labels)
print(assigments)
df["group"] = assigments
df

12
18
[np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0)]


,cluster,size,stratum,group
0,0,88,large,1
1,1,78,large,0
2,2,64,small,0
3,3,92,large,1
4,4,57,small,1
5,5,70,small,0
6,6,88,large,1
7,7,68,small,1
8,8,72,small,0
9,9,60,small,1


In [14]:
df[df["group"] == 0 ].size

60

In [15]:
df[df["group"] == 1 ].size

60

In [16]:
(df[(df["group"] == 0) & (df['stratum'] == 'small')].size)/(df[(df["group"] == 0) & (df['stratum'] == 'large')].size) 

1.5

In [17]:
(df[df['stratum'] == 'small'].size) / df[df['stratum'] == 'large'].size 

1.5

## 🔹 5. Re-Randomization / Constrained Randomization

Randomize, then check balance on covariates. If imbalance is too high, randomize again.
- Guarantees covariate balance.
- Used in constrained randomization designs.

In [37]:
n_clusters = 100
cluster_sizes = np.random.randint(50, 100, size = n_clusters)
threshold = 3

while True:
    assignments = np.random.choice([0, 1], size=n_clusters)
    label_1_mean = cluster_sizes[assignments == 1].mean()
    label_0_mean = cluster_sizes[assignments == 0].mean()
    if abs(label_1_mean -label_0_mean) <= threshold:
        break;
        

print(cluster_sizes)
print(assignments)



[94 83 96 89 72 73 78 87 63 88 97 98 67 98 82 88 66 86 60 79 55 69 76 73
 72 94 55 64 99 79 60 61 96 88 80 61 55 78 79 96 56 66 57 57 52 94 79 94
 99 69 51 59 59 90 70 85 73 51 52 51 82 81 78 56 76 64 97 96 74 65 91 55
 92 82 63 67 78 50 59 72 51 79 58 65 88 96 72 77 76 66 99 59 62 68 91 89
 51 90 60 85]
[1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 1
 1 1 0 1 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0
 1 1 0 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0]


### Multi-variate re-rendomization

In [ ]:
##-------TO-DO--------------

 ## 🔹 6. Cluster Randomization 

In [65]:
n_cluster = 20
sample_size = 500
df = pd.DataFrame({
        "student_id": np.arange(1, sample_size + 1),
        "cluster_id": np.repeat(np.arange(1, n_cluster + 1) , sample_size // n_cluster)
})
cluster_ids = df['cluster_id'].unique()
treatment_clusters = np.random.choice(cluster_ids, size=len(cluster_ids)//2, replace=False)
print(treatment_clusters)
df['group'] = df.cluster_id.apply(lambda c: "treatment" if c in treatment_clusters else "control")
print(df.head(100))


[18 13 11  2  6 17  9 14  4  8]
    student_id  cluster_id      group
0            1           1    control
1            2           1    control
2            3           1    control
3            4           1    control
4            5           1    control
..         ...         ...        ...
95          96           4  treatment
96          97           4  treatment
97          98           4  treatment
98          99           4  treatment
99         100           4  treatment

[100 rows x 3 columns]
